# Adaptive PDE discretizations on Cartesian grids
## Volume : Reproducible research
## Part : Comparison of acoustic and elastic schemes
## Chapter : Comparing dispersions, and error w.r.t exact solutions

In this notebook, we compare a several discretization schemes for the acoustic and elastic wave equations. The criteria are : 
- the faithfullness of the numerical dispersion relation, which is compared with the exact dispersion relation.
- the numerical error in synthetic test cases, obtained by diffeomorphic deformation of homogeneous media, and where an exact solution is known.

We compare a number of discretization schemes based on the Selling decomposition, with the baseline provided by the centered and crisscross schemes (Acoustic), and the Lebedev scheme for the elastic wave equation. These scheme implementations can be found in the notebooks on [comparisons of schemes for wave equations](ElasticComparisons.ipynb) and [high order methods for wave equations](HighOrderWaves.ipynb).

[**Summary**](Summary.ipynb) of volume Reproducible research, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Acoustic dispersion](#1.-Acoustic-dispersion)
    * [1.1 Two dimensions](#1.1-Two-dimensions)
    * [1.2 Three dimensions, needle-like anisotropy](#1.2-Three-dimensions,-needle-like-anisotropy)
    * [1.3 Three dimensions, plate-like anisotropy](#1.3-Three-dimensions,-plate-like-anisotropy)
  * [2. Elastic dispersion](#2.-Elastic-dispersion)
    * [2.1 Two dimensions](#2.1-Two-dimensions)
    * [2.2 Two dimensions, deformed materials](#2.2-Two-dimensions,-deformed-materials)
    * [2.3 Three dimensions](#2.3-Three-dimensions)
    * [2.4 Three dimensions, deformed materials](#2.4-Three-dimensions,-deformed-materials)
  * [3. Exact solutions in inhomogeneous domains](#3.-Exact-solutions-in-inhomogeneous-domains)
    * [3.1 Domain deformation](#3.1-Domain-deformation)
    * [3.2 Planewaves in the homogeneous domain](#3.2-Planewaves-in-the-homogeneous-domain)
    * [3.3 Inhomogeneous parameters and exact solution](#3.3-Inhomogeneous-parameters-and-exact-solution)
    * [3.4 Acoustic accuracy comparison](#3.4-Acoustic-accuracy-comparison)
    * [3.5 Elastic accuracy comparison](#3.5-Elastic-accuracy-comparison)



**Acknowledgement.** Some of the experiments presented in these notebooks are part of 
ongoing research with Ludovic Métivier and Da Chen.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('WaveAccuracy','Repro'))

In [2]:
from agd.ExportedCode.Notebooks_Div import ElasticComparisons as ec
from agd.ExportedCode.Notebooks_Div import HighOrderWaves as how
from agd.ExportedCode.Notebooks_Algo import TensorSelling as ts
from agd.Eikonal.HFM_CUDA import AnisotropicWave as aw
from agd import Metrics
from agd.Metrics import Riemann
from agd.Metrics.Seismic import Thomsen,Hooke
from agd import LinearParallel as lp
from agd import AutomaticDifferentiation as ad
from agd import FiniteDifferences as fd
from agd import Eikonal
from agd.Plotting import savefig; savefig.dirName = 'Images/WaveAccuracy/'
norm = ad.Optimization.norm

SyntaxError: invalid syntax (ElasticComparisons.py, line 46)

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from types import SimpleNamespace
π = np.pi
np.set_printoptions(linewidth=2000)

### 0.1 Utility functions

In [4]:
def norm_avg(arr,ord=2,axis=None,weights=1):
    """Averaged norm of an array, along the specified axes, with the specified weights"""
    # Normalize axis, if needed
    if axis is None: axis = tuple(range(arr.ndim))
    if np.ndim(axis)==0 and axis!=-1: axis=(axis,)
    if isinstance(axis,tuple):
        ndim = len(axis)
        arr = np.moveaxis(arr,axis,tuple(range(ndim)))
        weights = np.broadcast_to(weights,arr.shape[:ndim])
        weights = weights / np.sum(weights)
        arr = np.moveaxis(arr.reshape((-1,*arr.shape[ndim:])),0,-1)
        weights = weights.reshape(-1)
        axis=-1
    assert axis==-1
    if ord==np.inf: return np.max(arr,axis=-1)
    return np.sum(arr**ord * weights,axis=-1)**(1/ord)    

Compute a rotation which maps the vertical axis a given unit vector to another.

In [5]:
def rotation_params(u):
    """Compute the rotation mapping (0,1) onto u, assumed to be unit."""
    if len(u)==2: return np.arctan2(-u[0],u[1]) 
    ax = np.array((-u[1],u[0],np.zeros_like(u[0])))
    θ = np.arccos(u[2]/np.linalg.norm(u))
    return θ,ax/np.linalg.norm(ax)

In [6]:
u = np.array([-1.,2])
θ = rotation_params(u)
assert np.allclose(lp.rotation(θ) @ np.array([0,1.]), u/np.linalg.norm(u))
u = np.array([1.,2,-3])
θ,ax = rotation_params(u)
assert np.allclose(lp.rotation(θ,ax) @ np.array([0,0,1.]),u/np.linalg.norm(u))

In [7]:
def vti_orientations(nθ,vdim,prune=False):
    """A collection of orientations, and unit wavenumbers, with the given sampling density"""
    # Construction of orientations. We need to : 
    # - cover the projective space with the wavenumbers
    # - regarding anisotropy, and given the scheme invariances, it is sufficient to cover x>=y>=z>=0
    if vdim==2: 
        θ = np.linspace(0,π,nθ,endpoint=False)
        k = np.array((np.cos(θ),np.sin(θ))) # Cover y>=0 (projective space)
        θ = np.linspace(0,π/4,nθ//4) 
        uθ = np.array((np.cos(θ),np.sin(θ))) # Cover x>=y>=0 (part of first quadrant)
        weights=1
    elif vdim==3: # Three dimensional case
        θ = np.linspace(0,2*π,nθ*2, endpoint=False)[:,None]
        ϕ = np.linspace(0,π/2,nθ//2,endpoint=False)[None,:]
        k = np.array( (np.cos(θ)*np.cos(ϕ),np.sin(θ)*np.cos(ϕ),np.ones_like(θ)*np.sin(ϕ)) ) # Cover z>=0 (projective space)
        weights_k = np.cos(ϕ)+0.*θ
        k = k.reshape((3,-1)) 
        weights_k = weights_k.reshape(-1)
        select = np.logical_and(k[0]>=k[1],k[1]>=k[2])
        uθ = k[:,select] # Cover x>=y>=z>=0
        weights_θ = weights_k[select]
        # TODO : remove this. Not used anymore. 6D Voronoi decomposition, has been fixed
        if prune: uθ+=1e-3; uθ/=np.linalg.norm(uθ,axis=0); uθ=uθ[:,:44]; weights_θ=weights_θ[:44] 
        weights = weights_θ[:,None]*weights_k[None,:]
    return uθ,k,weights

In [8]:
def to_filename(str): 
    for key,val in (', ','-'),(' ','-'),('κ','kappa'): str = str.replace(key,val)
    return str

## 1. Acoustic dispersion

We compare the accuracy of the different discretizations, at a given number of points per wavelength.

### 1.1 Two dimensions 

Let us choose the parameters that we'll compare the schemes on.

In [9]:
def dispersion_exactAcoustic(k,ρ,D): return np.sqrt(lp.dot_VAV(k,D/ρ,k))

In [10]:
params_Acoustic2D = {
    'aniso':np.array([1,1.2,1.5,2,2.5,3,4,5,6,8,10,13]), # Anisotropy ratios
    'ppw':[3,4,5,6,8,10,12,15], # Points per wavelength

    'scheme':['centered2','centered4','centered6','crisscross2','crisscross4','Selling2','Selling4','Selling6'],
    'style':[':',':',':','--','--','-','-','-'], # Plot style, and color below
    'color':['blue','purple','cyan','green','olive','red','orange','brown'],
    'dispersion':[*(ec.dispersion_AcousticCentered,)*3,*(ec.dispersion_AcousticCrissCross,)*2,*(how.dispersion_a,)*3],
    'dispersion_exact':dispersion_exactAcoustic,
    'decomp':(lambda D:Eikonal.VoronoiDecomposition(D)), 'use_decomp':(False,)*5+(True,)*3, # Precompute Selling decomposition
    
    'ord':(1,2,np.inf), # norm order for the relative error 
    'ω_ref':'geom', # Alternatively : 'max', 'min', None
    'dt':1e-4, # Vanishingly small : only interested in spatial dispersion here

    'nθ':60, 'prune':False, # Angular discretization 
    'vdim':2,
    'mk_aniso':None, 'name':None, # To fill
    'mode':None,
}

In [11]:
def dispersions(default=params_Acoustic2D,**kwargs):
    """
    Compare the numerical dispersions of various schemes.
    Output: 
        - result : array with shape (ppw, scheme, error order, anisotropy type or parameter)
        - params : updated parameters
    """
    p = SimpleNamespace(**default) # parameters
    for key,val in kwargs.items(): assert key in dir(p); p.__setattr__(key,val)
    p.order_x = [int(scheme[-1]) for scheme in p.scheme]
    indices = [default['scheme'].index(scheme) for scheme in p.scheme]
    p.style = [p.style[i] for i in indices]
    p.color = [p.color[i] for i in indices]
    p.dispersion = [p.dispersion[i] for i in indices]

    uθ,k,weights = vti_orientations(p.nθ,p.vdim,p.prune)
    
    D_ = np.moveaxis(np.array([p.mk_aniso(uθ[...,None],aniso) for aniso in p.aniso]),0,2) # Last axis is for wavenumber k
    dt = p.dt # Fixed time step, often very small
    ρ = 1. # Fixed unit density
    shape = len(p.aniso),uθ.shape[1],k.shape[1]
    D_ = fd.as_field(D_,shape,singleton_in=True,singleton_out=True)

    # Choose a reference frequency, for which wavenumbers have norm approx 1
    def get_mode(x): return x if p.mode is None else x[p.mode]
    k_ = k[:,None,None,:]

    ω_exact = get_mode(p.dispersion_exact(k_,ρ,D_))
    ω_min,ω_max = np.min(ω_exact,axis=(1,2)),np.max(ω_exact,axis=(1,2))
    if p.ω_ref is not None:  # Normalize wavenumbers for the reference frequency
        ω_ref = {'min':ω_min,'max':ω_max,'geom':np.sqrt(ω_min*ω_max)}[p.ω_ref]
        k_ = k_* ω_ref[:,None,None]/ω_exact
        ω_exact = get_mode(p.dispersion_exact(k_,ρ,D_))

    result_ppw = []
    D_decomp = p.decomp(D_) # Precompute Voronoi's decomposition (actually useless, even in dim 6)
    for ppw in p.ppw:
        dx = 2*π/ppw # Points per wavelength
        result_scheme = []
        for dispersion,order_x,use_decomp in zip(p.dispersion,p.order_x,p.use_decomp):
            ω_scheme = get_mode(dispersion(k_,ρ,(D_decomp if use_decomp else D_),dx,dt,order_x)[0])
            rel_err = np.abs(1-ω_scheme/ω_exact)
            result_scheme.append([norm_avg(rel_err,ord,axis=(1,2),weights=weights) for ord in p.ord])
        result_ppw.append(result_scheme)

    return np.array(result_ppw),p

The following cell can take a little time to execute.

In [12]:
%%time
def mk_needle(uθ,κ): return Riemann.needle(uθ,1,κ).m
result_2d = dispersions(ppw=[4,6,10],mk_aniso=mk_needle,name='Two dimensional')

CPU times: user 16.2 ms, sys: 5.29 ms, total: 21.5 ms
Wall time: 26.1 ms


In [13]:
def compare_Acoustic(result,params,ppw=6,ord=2,anisos=None,schemes=None,styles=None):
    ippw = list(params.ppw).index(ppw)
    iord = list(params.ord).index(ord)
    ianiso = [list(params.aniso).index(aniso) for aniso in anisos] if anisos else np.arange(len(params.aniso))
    ischeme = [list(params.scheme).index(scheme) for scheme in schemes] if schemes else np.arange(len(params.scheme))
    if styles is None: styles = [params.style[i] for i in ischeme]
    
    plt.title(f"{params.name} dispersions, {ppw=}, {ord=}")
    plt.ylabel("Dispersion error")
    plt.xlabel("Anisotropy")
    for i,style in zip(ischeme,styles):
        plt.semilogy(params.aniso[ianiso],result[ippw,i,iord,ianiso],style,color=params.color[i],label=params.scheme[i])
    plt.legend()

We have the following observations:
- The centered schemes are accurate for isotropic, or weakly anisotropic models. It becomes terrible as soon as anisotropy exceeds two.
- The criss cross scheme is less accurate for isotropic models than the centered scheme, due to grid decoupling which effectively halves the resolution. It becompes more accurate for anisotropy exceeding $2$ typically.
- The selling scheme matches the accuracy of the centered scheme for isotropic metrics, and is the best one as soon as anisotropy exceeds 1.6 typically. There admittely is a small regime of weak anisotropy where it is beaten by the centered scheme.

The bottom line is that the fourth order Selling scheme is usable at 6ppw with any anisotropy, whereas the centered and criss-cross schemes would require increased sampling density.

In [14]:
compare_Acoustic(*result_2d)

In applications to seismic tomography, with topographic changes of coordinates, the anisotropy is unlikely to exceed $5$.
Previous observations still hold.

In [15]:
small_anisos = [κ for κ in result_2d[1].aniso if κ<=5]
compare_Acoustic(*result_2d,anisos=small_anisos)

A rule of thumb is to keep numerical dispersion around $10^{-2}$, which is typically achieved using :
- 10 points per wavelength for the second order scheme.
- 6 points per wavelength for the fourth order scheme.
- 4 (?) points per wavelength for the sixth order scheme.

Previous observations still hold.

In [16]:
plt.figure(figsize=[12,6])
plt.subplot(1,2,1); compare_Acoustic(*result_2d,anisos=small_anisos,ppw=10)
plt.subplot(1,2,2); compare_Acoustic(*result_2d,anisos=small_anisos,ppw=4)

We previously considered the mean square error on the numerical dispersion. 
Here we instead consider the worst case scenario instead. 
Observations : 
- The Selling scheme is now always best. The small region where it was beaten by the centered scheme, corresponding to weak anisotropy, has disappeared.
- The difference between the Selling and criss-cross scheme is a bit smaller than before.

In [17]:
compare_Acoustic(*result_2d,anisos=small_anisos,ord=np.inf)

The sixth order schemes are a bit of a stretch. Usually, one only consider second and fourth order.

In [18]:
compare_Acoustic(*result_2d,anisos=small_anisos,ord=np.inf,
                 schemes=['centered2','centered4','crisscross2','crisscross4','Selling2','Selling4'])

### 1.2 Three dimensions, needle-like anisotropy

For simplicity, we'll focus on two types of anisotropies : needle like, and plate like. 
- needle-like : eigenvalues are $(\kappa, 1, 1)$, where $\kappa > 1$. Common in image processing, motion planning, etc
- plate-like : eigenvalues are $(1,\kappa,\kappa)$, where $\kappa>1$. Natural in seismic applications, since waves travels faster horizontally than vertically in layered media.

In [19]:
%%time
# May take up to a minute to execute
result_needle = dispersions(ppw=[6,10],vdim=3,nθ=40,mk_aniso=mk_needle,name="Needle-like 3d")

CPU times: user 1.28 s, sys: 633 ms, total: 1.91 s
Wall time: 1.95 s


**Needle case**

The Selling schemes, and the second order crisscross scheme, do not appear to suffer from anisotropy at all.

In [20]:
compare_Acoustic(*result_needle)

Very similar conclusions to the two dimensional case.

In [21]:
compare_Acoustic(*result_needle,anisos=small_anisos,ord=np.inf,
                 schemes=['centered2','centered4','crisscross2','crisscross4','Selling2','Selling4'])

In [22]:
compare_Acoustic(*result_needle,anisos=small_anisos,ppw=10,
                 schemes=['centered2','centered4','crisscross2','crisscross4','Selling2','Selling4'])

### 1.3 Three dimensions, plate-like anisotropy

The results are very similar to the two-dimensional and needle-like cases.

In [23]:
%%time
# May take up to a minute to execute
def mk_plate(uθ,κ): return Riemann.needle(uθ,κ,1).m
result_plate = dispersions(ppw=[6,10],vdim=3,nθ=40,mk_aniso=mk_plate,name="Plate-like 3d")

CPU times: user 1.36 s, sys: 722 ms, total: 2.08 s
Wall time: 2.13 s


In [24]:
compare_Acoustic(*result_plate)

In [25]:
compare_Acoustic(*result_plate,anisos=small_anisos,ord=np.inf,
                 schemes=['centered2','centered4','crisscross2','crisscross4','Selling2','Selling4'])

In [26]:
# Additional image exports
for result in (result_2d,result_needle,result_plate):
    for ord in (2,np.inf):
        fig = plt.figure()
        compare_Acoustic(*result,anisos=small_anisos,ord=ord,schemes=['centered2','centered4','crisscross2','crisscross4','Selling2','Selling4'])
        savefig(fig,to_filename(result[1].name)+f"-{ord=}.png")
        plt.close(fig)

## 2. Elastic dispersion

Elastic waves have $d$ propagation modes in dimension $d$. We sort them from slowest to fastest. The fastest mode is often referred to as the pressure wave, while the others are shear waves.

### 2.1 Two dimensions

<!---
ϵ=1e-6; ω_exact,_,_ = dispersion_Virieux(ks,ρ,C,ϵ*dx,ϵ*dt,order_x=order_x) # Too lazy
#ω_Virieux2,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=2)
#ω_Virieux4,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=4)
ω_Lebedev2,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=2)
ω_Lebedev4,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=4)

#ω_Selling2,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=2,order_t=2)
#ω_Selling4,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=4,order_t=2)
ω_Selling2_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=2)
ω_Selling4_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=4)

ω_Selling2_stag,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=2)
ω_Selling4_stag,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=4)

ω_Selling2_stagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=2,decomp=staggered_decomp_linprogExt)
ω_Selling4_stagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=4,decomp=staggered_decomp_linprogExt)

ω_Selling2_mystagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=2,decomp=mydecomp)
ω_Selling4_mystagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=4,decomp=mydecomp)

#ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=6,order_t=2) # Order 6 does not bring much benefit
#ω_cont,_ = Hooke(Cs).waves(ks,ρs) #lp.dot_VAV(ks,Ds,ks)/ρs
--->

In [27]:
def dispersion_exactElastic(k,ρ,C): 
    shape = tuple(np.maximum(k.shape[1:],C.shape[2:]))
    k = np.broadcast_to(k,(*k.shape[:1],*shape))
    C = np.broadcast_to(C,(*C.shape[:2],*shape))
    ω2 = np.moveaxis(np.linalg.eigvalsh(np.moveaxis(Hooke(C/ρ).contract(k),(0,1),(-2,-1))),-1,0)
    return np.sqrt(ω2)
#    return Hooke(C).waves(k,ρ)[0]
def dispersion_Lebedev(k,ρ,C,dx,dt,order_x):
    s2 = 2**(1-1/len(k))
    return ec.dispersion_Virieux(k,ρ,C,dx*s2,dt,order_x)
def dispersion_SellingStaggeredExt2(k,ρ,C,dx,dt,order_x): 
    return ec.dispersion_SellingStaggered2(k,ρ,C,dx,dt,order_x,decomp=ec.staggered_decomp_linprogExt)

In [28]:
params_elastic_Thomsen2 = {
    'aniso':list(Thomsen.ThomsenData.keys())+['mica','olivine','stishovite'], # Anisotropy types
    'ppw':[6,10], #[3,4,5,6,8,10,12,15], # Points per wavelength

    'scheme':['Virieux2','Virieux4','Lebedev2','Lebedev4','Selling2','Selling4','Selling6',
              'Selling_corr2','Selling_corr4','Selling_corr6','Selling_stag2','Selling_stag4','Selling_stagExt2','Selling_stagExt4'],
    'style':[':',':', '--','--', '-','-','-', '-.','-.','-.', 'o','o', 'x','x'], # Plot style, and color below
    'color':['blue','cyan', 'green','olive', 'red','orange','brown', 'red','orange','brown', 'pink','purple', 'pink','purple'],
    'dispersion':[*(ec.dispersion_Virieux,)*2,*(dispersion_Lebedev,)*2,*(how.dispersion_e,)*3,*(ec.dispersion_SellingCorrelated,)*3,
                 *(ec.dispersion_SellingStaggered2,)*2,*(dispersion_SellingStaggeredExt2,)*2],
    'dispersion_exact':dispersion_exactElastic,
    'decomp':(lambda D:Hooke(D).Selling()), 'use_decomp':(False,)*4+(True,)*6+(False,)*4, # Precompute Selling decomposition

    'ord':(1,2,np.inf), # norm order for the relative error 
    'ω_ref':'geom', # Alternatively : 'max', 'min', None
    'dt':1e-4, # Vanishingly small : only interested in spatial dispersion here

    'nθ':60,'prune':False, # Angular discretization 
    'vdim':2,
    'mk_aniso':None, 'name':None, 'mode':None, # To fill
}

In [29]:
def hooke_vti_rotate(uθ,hk):
    rot = rotation_params(uθ) # vdim==2 : angle. vdim==3 : angle and axis
    return (hk.rotate_by(rot) if len(uθ)==2 else hk.rotate_by(*rot)).hooke # Rotate and export
    
def mk_Thomsen(uθ,name):
    """Rotate the elastic material designed by name, so that the vertical axis is uθ"""
    if name in Thomsen.ThomsenData: hk = Hooke.from_ThomsenElastic(Thomsen.ThomsenData[name])[0] # Thomsen materials
    else: hk = {'mica':Hooke.mica, 'olivine':Hooke.olivine, 'stishovite':Hooke.stishovite}[name][0]
    if len(uθ)==2: hk = hk.extract_xz()
    hk.hooke/=np.max(hk.hooke) # Normalize
    return hooke_vti_rotate(uθ,hk)

The next two cells may take a minute to execute.

In [30]:
%%time
result_pressure_Thomsen2 = dispersions(default=params_elastic_Thomsen2,mk_aniso=mk_Thomsen,name='Pressure Thomsen 2d',mode=1)

CPU times: user 1.2 s, sys: 331 ms, total: 1.54 s
Wall time: 1.56 s


In [31]:
%%time
result_shear_Thomsen2 = dispersions(default=params_elastic_Thomsen2,mk_aniso=mk_Thomsen,name='Shear Thomsen 2d',mode=0)

/Users/jean-mariemirebeau/Dropbox/Programmes/GithubM1/AdaptiveGridDiscretizations/Notebooks_Repro/../../AdaptiveGridDiscretizations/agd/ExportedCode/Notebooks_Div/ElasticComparisons.py:630: RuntimeWarning: invalid value encountered in sqrt
  Iω = np.sqrt(Iω2)


CPU times: user 1.19 s, sys: 300 ms, total: 1.49 s
Wall time: 1.52 s


Not sure how to plot these results. We can first look at how often each scheme is first. Or compare to the reference dispersion, e.g. Lebedev ? Likely better to compare within a given order class, or stencil width.

We may also want to sort the materials by difficulty. Based e.g. on the dispersion of the Lebedev scheme ? 

In [32]:
def compare_elastic_Thomsen(result,params,ppw=6,ord=2,anisos=None,schemes=None,styles=None):
    ippw = list(params.ppw).index(ppw)
    iord = list(params.ord).index(ord)
    ianiso = [list(params.aniso).index(aniso) for aniso in anisos] if anisos else np.arange(len(params.aniso))
    ischeme = [list(params.scheme).index(scheme) for scheme in schemes] if schemes else np.arange(len(params.scheme))
    if styles is None: styles = [params.style[i] for i in ischeme]
    
    plt.title(f"{params.name} dispersions, {ppw=}, {ord=}")
    plt.ylabel("Dispersion error")
    plt.xlabel("Anisotropic material")
    ax = params.aniso if isinstance(params.aniso,np.ndarray) else np.arange(len(params.aniso))
    for i,style in zip(ischeme,styles):
        plt.semilogy(ax,result[ippw,i,iord,ianiso],style,color=params.color[i],label=params.scheme[i])
    plt.legend()

Looking at the pressure wave data : 
- The Selling schemes, from best to worst, appear to be usually : correlated, standard, staggered, staggered_ext.
- There are a few circumstances in which the order is reversed. In particular, the staggered schemes are better for pressure and order two.
- The Selling schemes have annoying spikes in anisotropy. The spikes can also be seen in the Virieux and Lebedev schemes, but are much less strong.
- The spikes are a little less strong with the correlated scheme.
  
We should figure out what causes these spikes. (Type of material, Selling offsets.)

In [33]:
fig = plt.figure(figsize=(14,6))
result = result_pressure_Thomsen2
compare_elastic_Thomsen(*result,ppw=6)
savefig(fig,to_filename(result[1].name)+'-all.png')

Looking at the shear wave data.

We have the following observations:
- we often beat the non-existing Virieux scheme, for mildly anisotropic materials.
- There are some very pronounced spikes, for very anisotropic materials.

Among Selling schemes : 
- *fourth order*, from best to worst : correlated, standard, staggered, staggered ext. However there are more inversions than for the pressure wave, and staggered is often better.
- *second order*, from best to worts : staggered, staggered ext, correlated, standard.

In [34]:
fig = plt.figure(figsize=(14,6))
result = result_shear_Thomsen2
compare_elastic_Thomsen(*result,ppw=6)
savefig(fig,to_filename(result[1].name)+'-all.png')

Let us try to figure out what type of material is creating these bad spikes, defavorable to the Selling schemes.
If we focus on the fourth order schemes, which appear to be most popular, the bad guys are mostly crystals.

In other words, our schemes suffer from strong anisotropy, which is a bit surprising in view of the acoustic results.

In [35]:
iLebedev = params_elastic_Thomsen2['scheme'].index('Lebedev4')
iSelling_corr = params_elastic_Thomsen2['scheme'].index('Selling_corr4')
ippw = 0 # 6 ppw
iord = 1 # L^2 err
res = result_shear_Thomsen2[0][ippw,:,iord,:] 
bad = res[iLebedev]<res[iSelling_corr]
print(f"Number of bad materials {np.sum(bad)}")
bad_materials = [params_elastic_Thomsen2['aniso'][i] for i,b in enumerate(bad) if b]
bad_materials

Number of bad materials 6


['Wills Point shale - 1',
 'Muscovite crystal',
 'Biotite crystal',
 'Aluminium-lucite composite',
 'Gypsum-weathered material',
 'mica']

In [36]:
# Additional image exports
for result in (result_pressure_Thomsen2,result_shear_Thomsen2):
    for ord in (2,np.inf):
        fig = plt.figure(figsize=[14,6])
        compare_elastic_Thomsen(*result,ord=ord,schemes=('Lebedev2','Lebedev4','Selling_corr2','Selling_corr4'),
                                styles=[':',':','-','-'],ppw=10)
        savefig(fig,to_filename(result[1].name)+f"-Leb-corr-{ord=}.png")
        plt.close(fig)

### 2.2 Two dimensions, deformed materials

Some anisotropy arises naturally, by the chemical structure of the material. But in other contexts anisotropy arises artificially from changes of coordinates in the domain. 

Here we try to looke at the dependency of the dispersion to this type of anisotropy. 

**Coefficients of the deformed elastic wave equation.**
Changes of coordinates affect several coefficients in the elastic wave equation:
- the hooke tensor, as considered here.
- the symmetric positive definite matrix related to the kinetic energy. It is *not considered here*, and as a result the dispersion relation is *inexact*. 
- a low order additive term to the strain tensor, which only arises for non-linear changes of coordinates, and is not considered here.

Correct theoretical and numerical dispersion relations with anisotropic kinetic energy are doable but require a bit more work.

**Effect on the Hooke tensor.**
In the current state, material deformations appears *very bad* for the Selling schemes. The underlying reason is the following : define the anisotropy ratio
$$
    \mu(D) := \sqrt{\lambda_{\max}(D)/\lambda_{\min}(D)}
$$
of a symmetric positive definite matrix. Consider now a physical material to which a dilation $\kappa_*$ is applied along some axis. Then : 
- In the acoustic case, anisotropy $\mu(D_*)\approx \kappa_*$ of the resulting tensor scales linearly with $\kappa_*$.
- In the elastic case, the Hooke tensor anisotropy $\mu(C_*) \approx \kappa_*^2$ scales quadratically with $\kappa_*$.

The Selling offsets, on the other hand, sale approximately linearly with the tensor anisotropy $\mu(D_*)$ or $\mu(C_*)$. 
This means that the Selling schemes will use excessively large offsets, as soon as anisotropy is a bit pronounced, which strongly degrades their effective accuracy.

All the Selling schemes are subject to this behavior. 
- The staggered schemes quickly become undefined, because we only classified a few vertices of the equivalent linear program. The other schemes remain well defined, but dispersion becomes horrendous.
- High order schemes do not really help, and sometimes even degrade the situation.

The bottom line is that the Selling schemes should likely be rejected for distortions with anisotropy ratios $\geq 2$. Sadly, no extreme anisotropy can be envisionned, contrary to the acoustic case. For weaker deformations, they remain usable. 

<!---
Erroneous comments, from when pressure and shear modes were exchanged.
- The only positive point is that the pressure wave seems worse than the shear wave (In applications, the shear wave dispersion is limiting, since the corresponding wavelength is shorter). In the shear case, the correlated Selling scheme remains competitive with Lebedev up to anisotropy ratios $\approx 5$.

 taking into account that the shear wave dispersion is the bottlenect,
--->

In [37]:
params_elastic_needle2 = params_elastic_Thomsen2.copy()
params_elastic_needle2['aniso']=np.array([1,1.2,1.5,2,2.5,3,4,5,6]) # Deformation ratios # 8,10,13

In [38]:
hk_ref = Hooke.stishovite[0].extract_xz()
hk_ref.hooke/=np.max(hk_ref.hooke)
def mk_elastic_needle(uθ,κ):
    """Deform the elastic medium, with 1/κ along horizontal axis, then rotate."""
    diag = np.diag((κ,)+(1,)*(len(uθ)-1))
    return hooke_vti_rotate(uθ,hk_ref.rotate(diag)) # Not actually a rotation...
def mk_elastic_plate(uθ,κ):
    """Deform the elastic medium, with 1/κ along horizontal axis, then rotate."""
    diag = np.diag((1,)+(κ,)*(len(uθ)-1))
    return hooke_vti_rotate(uθ,hk_ref.rotate(diag)) # Not actually a rotation...

In [40]:
uθ = np.array((1.,0))
hk = mk_elastic_needle(uθ,10)
Hooke(hk).norm(uθ),Hooke(hk).norm(lp.perp(uθ))

(0.17548119783512647, 0.013088255355702327)

Here we can see that a deformation by a factor $10$ along one axis, implies a ratio $\approx 10^4$ on the eigenvalues.

In [41]:
np.linalg.eigvalsh(mk_elastic_needle((1,0),10))

array([8.82754055e-01, 3.24742268e+01, 5.83774611e+03])

In [42]:
%%time
result_pressure_needle2 = dispersions(default=params_elastic_needle2,mk_aniso=mk_elastic_needle,name='Pressure needle 2d',mode=1)

CPU times: user 212 ms, sys: 42.4 ms, total: 255 ms
Wall time: 256 ms


In [43]:
%%time
result_shear_needle2 = dispersions(default=params_elastic_needle2,mk_aniso=mk_elastic_needle,name='Shear needle 2d',mode=0)

CPU times: user 197 ms, sys: 39.9 ms, total: 237 ms
Wall time: 238 ms


In [ ]:
fig = plt.figure(figsize=[14,6])
result = result_pressure_needle2
compare_elastic_Thomsen(*result,ppw=10)
savefig(fig,to_filename(result[1].name)+"-all.png")

In [ ]:
fig = plt.figure(figsize=[14,6])
result = result_shear_needle2
compare_elastic_Thomsen(*result) 
savefig(fig,to_filename(result[1].name)+"-all.png")

**Additional image exports**

In [ ]:
for result in (result_pressure_needle2,result_shear_needle2):
    for ord in (2,np.inf):
        fig = plt.figure(figsize=[14,6])
        compare_elastic_Thomsen(*result,ord=ord,schemes=('Lebedev2','Lebedev4','Selling_corr2','Selling_corr4'),styles=[':',':','-','-'])
        savefig(fig,to_filename(result[1].name)+f"-Leb-corr-{ord=}.png")
        plt.close(fig)

### 2.3 Three dimensions

In three dimensions, only the standard variant of the elastic scheme is currently implemented. 
Implementing the correlated variant should not be too hard. The staggered variants are more complicated to generalize, but in view of the previous results there is no need to rush.

<!---
Over the 52 orientations, there are only a few that raise issues typically. We prune them out.
 #,prune=True)
--->

In [ ]:
uθ,_,_ = vti_orientations(40,3)
bad_decomp = []
for name in params_elastic_Thomsen2['aniso']:
    C = mk_Thomsen(uθ,name)
    λ,e = Eikonal.VoronoiDecomposition(C)
    bad_λ = np.any(np.isnan(λ),axis=0)
    if np.any(bad_λ): 
        bad_decomp.append((name,np.sum(bad_λ)))
        print(np.nonzero(bad_λ))

In [ ]:
assert len(bad_decomp)==0
# 6D Voronoi decomposition was fixed by adding a tolerance for the positivity of coefficients.

In [ ]:
params_elastic_Thomsen3 = params_elastic_Thomsen2.copy()
params_elastic_Thomsen3.update({
    'nθ':20, # 40, yields 13 min computation time, accuracy not much improved
    'vdim':3,
    'aniso':[name for name in params_elastic_Thomsen2['aniso'] if name not in bad_decomp],
    'dispersion':params_elastic_Thomsen2['dispersion'][:10],
    'scheme':params_elastic_Thomsen2['scheme'][:10],
    'prune':True,
})

The following cells take a minute to execute each.

In [ ]:
%%time
result_pressure_Thomsen3 = dispersions(default=params_elastic_Thomsen3,mk_aniso=mk_Thomsen,name='Pressure Thomsen 3d',mode=2)

In [ ]:
%%time
result_shear1_Thomsen3 = dispersions(default=params_elastic_Thomsen3,mk_aniso=mk_Thomsen,name='Fast shear Thomsen 3d',mode=1)

In [ ]:
%%time
result_shear0_Thomsen3 = dispersions(default=params_elastic_Thomsen3,mk_aniso=mk_Thomsen,name='Slow shear Thomsen 3d',mode=0)

The situation is more or less comparable to the two dimensional case : the Selling scheme is usually better than the Lebedev scheme, except for a few crystals where the dispersion spikes. 

The correlated scheme improves things a bit:
- mostly for the pressure wave, not as much for the shear waves.
- the improvement is less strong than in two dimensions.

In [ ]:
fig = plt.figure(figsize=[14,6])
compare_elastic_Thomsen(*result_pressure_Thomsen3)
savefig(fig,"Pressure-Thomsen-3d-all.png")

From the dispersion relations of the shear waves, we observe that:
- Spikes are a bit less hard on the shear waves.
- The Selling scheme is on par with the Virieux scheme for the first shear wave, but somewhat worse for the slowest one.

In [ ]:
fig = plt.figure(figsize=[14,6])
compare_elastic_Thomsen(*result_shear1_Thomsen3)
savefig(fig,"Shear1-Thomsen-3d-all.png")

In [ ]:
fig = plt.figure(figsize=[14,6])
compare_elastic_Thomsen(*result_shear0_Thomsen3)
savefig(fig,"Shear0-Thomsen3d-all.png")

**Saving a few additional figures**

In [ ]:
for result in (result_pressure_Thomsen3,result_shear1_Thomsen3,result_shear0_Thomsen3):
    for ord in (2,np.inf):
        fig = plt.figure(figsize=[14,6])
        compare_elastic_Thomsen(*result,ord=ord,schemes=('Lebedev2','Lebedev4','Selling_corr2','Selling_corr4'),styles=[':',':','-','-'])
        savefig(fig,to_filename(result[1].name)+f"-Leb-corr-{ord=}.png")
        plt.close(fig)

### 2.4 Three dimensions, deformed materials

This subsection is *todo*, but first we should obtain the correct dispersion relations under changes of variables.

## 3. Exact solutions in inhomogeneous domains

We compare the accuracy of the two dimensional schemes for acoustic and elastic wave equations. For that purpose, we design a synthetic test case with periodic boundary conditions and an exact solution.

### 3.1 Domain deformation

The synthetic inhomogeneous test case is obtained as a diffeomorphic deformation of a homogeneous domain. 
The chosen deformation is smooth, periodic, and otherwise rather arbitrary. A parameter allows to tune the strength of the deformation.

In [ ]:
def ϕ_default(X,ϵ=0.05): 
	"""Some perturbation of the identity map on the torus (R/Z)^2. Appears to be invertible when ϵ<=0.05"""
	x,y = X
	return ad.array([x + 0.6*ϵ*np.sin(2*π*(x+2*y)+2) + ϵ*np.sin(2*π*x), 
					 y - ϵ*np.exp(np.cos(2*π*(x-y))) ])

In order to visualize the deformation, we apply it to a regular grid.

In [ ]:
plt.figure(figsize=[12,6])
for i,ϵ in enumerate((0.02,0.05)):
    plt.subplot(1,2,1+i)
    plt.title(f"Deformed medium, {ϵ=:.2f}")
    X,dx = how.make_domain(20,2)
    plt.scatter(*ϕ_default(X,ϵ))
    plt.axis('equal');

The singular values of the jacobian quantify the amount of distorsion. A substantial deformation already occurs for $\epsilon=0.02$, with a ratio $\approx 1.66$ of the dilations along orthogonal axes.

In [ ]:
X_ad = ad.Dense.identity(constant=X,shape_free=(2,))
for ϵ in (0.01,0.02,0.05):
    dϕ = ϕ_default(X_ad,ϵ).gradient()
    S = np.moveaxis(np.linalg.svd(np.moveaxis(dϕ,(0,1),(-2,-1))).S,-1,0)
    print(f"Max deformation ratio for {ϵ=:.2f} : ", np.max(S[0]/S[1]))

### 3.2 Planewaves in the homogeneous domain

The exact solution in the inhomogeneous deformed domain is obtained, appropriately, as a diffeomorphic deformation of an exact solution in the original homogeneous domain.
The latter is defined as a sum of planewaves, in all directions, with random phases and amplitudes.


In [ ]:
def rand_cos():
	"""Returns a cosine-like 1-periodic function with a random phase and amplitude."""
	amplitude, phase = np.random.rand(2)
	return lambda s : amplitude*np.cos(2*π*(s+phase))

Planewaves in all directions, with random amplitudes, phases, and propagation modes in the elastic case. 

*Periodicity.* The modulating functions are $1$-periodic, rather than $2\pi$ periodic. The variable `kr` below stands for *reduced wave number*, and the actual wavenumber is $2 \pi \times$ `kr`.

In [ ]:
np.random.seed(42)
krs = [(i,j) for i in range(-2,3) for j in range(-2,3) if not (i in (-2,0,2) and j in (-2,0,2))]
planewaves_default_a = [ (kr, rand_cos() ) for kr in krs]
planewaves_default_e = [ (kr, rand_cos(), mode) for kr in krs for mode in range(2)]

In [ ]:
def planewave_with_spatial_frequency(Nwaves,planewave):
	"""Adjust the number of oscillations in a planewave, within the domain [-1,1]^d, retaining periodicity"""
	kred,fun = planewave[:2]
	return (kred,lambda s:fun(s*np.floor(Nwaves/np.linalg.norm(kred))))+planewave[2:]

In [ ]:
Nwaves = 8
Nx=100
X,dx = how.make_domain(Nx,2)
kr,f = planewave_with_spatial_frequency(Nwaves,planewaves_default_a[0])
plt.contourf(*X,f(kr[0]*X[0]+kr[1]*X[1]))
plt.axis('equal');

### 3.3 Inhomogeneous parameters and exact solution

The transformations of the coefficients and solution associated to a domain diffeomorphism are described in detail in [the notebook on high order schemes for wave equations](../Notebook_Div/HighOrderWaves.ipynb).

In [ ]:
def make_test_a(ρ_flat,D_flat,X,dx, ϕ=ϕ_default):
    """Parameters and exact solution of the acoustic wave equation in a deformed homogeneous domain"""
    from agd.ExportedCode.Notebooks_Div.HighOrderWaves import ExactSol_a,tq_a,tp_a,tρ_a,tD_a
    ϕ,dϕ,inv_dϕ,Jϕ,d2ϕ = how.differentiate(lambda x:ϕ(x),X)
    ρ = tρ_a(lambda x:ρ_flat,ϕ,Jϕ)
    D = tD_a(lambda x:D_flat,ϕ,inv_dϕ,Jϕ)
    def make_sol(planewaves=planewaves_default_a):
        """Exact homogeneous planewave solution, deformed by ϕ."""
        q_flat,p_flat = ExactSol_a(ρ_flat,D_flat,*zip(*planewaves))
        def q(t): return tq_a(lambda x:q_flat(t,x),ϕ)
        def p(t): return tp_a(lambda x:p_flat(t,x),ϕ,Jϕ)
        return q,p
    return ρ,D,make_sol

In [ ]:
def make_test_e(M_flat,C_flat,X,dx, ϕ=ϕ_default): 
    """Parameters and exact solution of the elastic wave equation in a deformed homogeneous domain"""
    from agd.ExportedCode.Notebooks_Div.HighOrderWaves import ExactSol_e,tq_e,tp_e,tM_e,tC_e,tS_e
    ϕ,dϕ,inv_dϕ,Jϕ,d2ϕ = how.differentiate(lambda x:ϕ(x),X)
    M = tM_e(lambda x:M_flat,ϕ,dϕ,Jϕ)
    C = tC_e(lambda x:C_flat,ϕ,inv_dϕ,Jϕ)
    vdim=len(X); S_flat = np.zeros((vdim,vdim,vdim))
    S = tS_e(lambda x:S_flat,ϕ,dϕ,inv_dϕ,d2ϕ)
    def make_sol(planewaves=planewaves_default_e):
        """Exact homogeneous planewave solution, deformed by ϕ."""
        q_flat,p_flat = ExactSol_e(M_flat,C_flat,*zip(*planewaves))
        def q(t): return tq_e(lambda x:q_flat(t,x),ϕ,dϕ)
        def p(t): return tp_e(lambda x:p_flat(t,x),ϕ,inv_dϕ,Jϕ)
        return q,p
    return M,C,S,make_sol

**Acoustic test case with strong anisotropy.**

In [ ]:
ϕ_acoustic = lambda X:ϕ_default(X,0.05)
ρ_flat = 1.
D_flat = Riemann.from_diagonal([3**2, 1]).rotate_by(π/8).m
Nx = 150
X,dx = how.make_domain(Nx,2)
ρ,D,_ = make_test_a(ρ_flat,D_flat,X,dx,ϕ_acoustic)

Because of the strong anisotropy, the linear decomposition presented in [the notebook on Selling's decomposition](../Notebooks_Div/TensorSelling.ipynb) has negative coefficients already for the homogeneous medium (without additional deformation).

In [ ]:
ts.linear_decomp(D_flat)

Furtunately the tensor field is sufficiently smooth for applying the deconvolution method. (I.e. the deconvolved field remains positive definite.) Note that the number of positive coefficients exceeds the Selling case ($3$ coefficients).

*Deconvolution width parameter $\sigma$ (measured in pixels).*
Here we need to reduce the value of this parameter, to ensure that the deconvolved matrices remain positive definite.

In [ ]:
λ,e = ts.conv_decomp(*ts.deconv(D,σ=2.5,depth=2)) # Always use depth=2 
print(f"Maximum number of positive coefficients {len(λ)}")
print(f"Average number of positive coefficients {np.mean(np.sum(λ>0,axis=0))}")

Compare with the smooth variant of Selling's two-dimensional decomposition.

In [ ]:
λ,e = Eikonal.VoronoiDecomposition(D,smooth=True) # Always use depth=2 
print(f"Maximum number of positive coefficients {len(λ)}")
print(f"Average number of positive coefficients {np.mean(np.sum(λ>0,axis=0))}")

**Elastic test case with mild anisotropy.**

In [ ]:
ϕ_elastic = lambda X:ϕ_default(X,0.01)
M_flat = np.eye(2)
C_flat = Hooke.olivine[0].extract_xz().rotate_by(π/8).hooke; C_flat/=np.max(C_flat)
Nx = 150
X,dx = how.make_domain(Nx,2)
M,C,_,_ = make_test_e(M_flat,C_flat,X,dx,ϕ_elastic)

Again the, the linear decomposition is already non-positive in the homogeneous medium.

In [ ]:
ts.linear_decomp(C_flat)

Since `ϕ_elastic` is smoother than `ϕ_acoustic`, in view of the smaller parameter $\epsilon$, we can increase the size of the convolution support $\sigma$. 

In [ ]:
λ,e = ts.conv_decomp(*ts.deconv(C,σ=5,ϵ=2e-4,depth=2)) 
print(f"Maximum number of positive coefficients {len(λ)}")
print(f"Average number of positive coefficients {np.mean(np.sum(λ>0,axis=0))}")

Compare with the smooth variant of Selling's decomposition.

In [ ]:
λ,e = Eikonal.VoronoiDecomposition(C,smooth=True)
print(f"Maximum number of positive coefficients {len(λ)}")
print(f"Average number of positive coefficients {np.mean(np.sum(λ>0,axis=0))}")

### 3.4 Acoustic accuracy comparison

We compare the accuracy of different schemes for the acoustic wave equation. Time integration relies on the Euler symplectic method, which is second order in time if viewed with the appropriate half-step time shifts.

**Scheme order in space.** 
Staggered schemes are implemented up to order 4, whereas centered schemes are implemented up to order 6. 
(Note : in the one dimension case, a staggered scheme of order 4, and a centered scheme of order 6, both yield a 7 point finite difference approximation of the laplacian.)

**Points per wavelength (ppw).** 
The given number of points per wavelength assumes no domain deformation. It should be regarded as an average value. The local number of ppw depends on the distortion induced by the diffeomorphism, which can be significant. 

**Smooth variants of Selling's decomposition.**
The standard Selling scheme are very accurate for low ppw. However, for high ppw, in the $L^\infty$ norm, it does not appear to converge at the expected order (consistency order of the Hamiltonian discretization). This is likely due to the non-smoothness of the coefficients, and it is why we also consider variants of Selling's decomposition with smooth coefficients for comparison.

In [ ]:
ϵ_acoustic = 0.05; ϕ_acoustic = lambda X:ϕ_default(X,ϵ_acoustic)
ρ_flat = 1.
D_flat = Riemann.from_diagonal([3**2, 1]).rotate_by(π/8).m
Nt = 100 # Number of time steps 

In [ ]:
%%time
Nxs = [150,250] # Considered domain resolutions
ppws = [7,8,10,12,15,20] # ppw in flat domain. TODO : take deformation into account.

results_Nx = []
for Nx in Nxs:
    # Build the domain, and coefficients
    X,dx = how.make_domain(Nx,2)
    ρ,D,make_sol = make_test_a(ρ_flat,D_flat,X,dx,ϕ=ϕ_acoustic)

    # Build the hamiltonians
    stag2 = ec.staggered(dx,2,'Periodic')
    stag4 = ec.staggered(dx,4,'Periodic')
    sel_D = Eikonal.VoronoiDecomposition(D)
    smooth_D = ts.smooth_decomp(D,order=3)
    conv_D = ts.conv_decomp(*ts.deconv(D,σ=2.5,ϵ=1e-2,depth=2)) # order=7 previously
#    lin_D = ts.linear_decomp(D) # Non-positive for this anisotropy

    # Set a timestep
    WaveH = aw.AcousticHamiltonian_Sparse(ρ,sel_D,dx,2)
    dt = 0.5*WaveH.dt_max()
    T = Nt*dt

    WaveHs = [ # All Hamiltonians
        ("CrissCross2",ec.AcousticCrissCrossH(ρ,D,X,dx,stag2)),
        ("CrissCross4",ec.AcousticCrissCrossH(ρ,D,X,dx,stag4)),
        ("Centered2",ec.AcousticCenteredH(ρ,D,X,dx,2)),
        ("Centered4",ec.AcousticCenteredH(ρ,D,X,dx,4)),
#        ("Centered6",ec.AcousticCenteredH(ρ,D,X,dx,6)),
        ("Selling2",WaveH),
        ("Selling4",aw.AcousticHamiltonian_Sparse(ρ,sel_D,dx,4)),
#        ("Selling6",aw.AcousticHamiltonian_Sparse(ρ,sel_D,dx,6)),
        ("ConvSelling2",aw.AcousticHamiltonian_Sparse(ρ,conv_D,dx,2)),
        ("ConvSelling4",aw.AcousticHamiltonian_Sparse(ρ,conv_D,dx,4)),
#        ("ConvSelling6",aw.AcousticHamiltonian_Sparse(ρ,conv_D,dx,6)),
        ("SmoothSelling2",aw.AcousticHamiltonian_Sparse(ρ,smooth_D,dx,2)),
        ("SmoothSelling4",aw.AcousticHamiltonian_Sparse(ρ,smooth_D,dx,4)),
#        ("SmoothSelling6",aw.AcousticHamiltonian_Sparse(ρ,smooth_D,dx,6)),
         ]
     
    results_ppw = []
    for ppw in ppws:
        print(f"{Nx=}, {ppw=}")
        # Build an exact solution
        planewaves = [planewave_with_spatial_frequency(Nx/ppw,planewave) for planewave in planewaves_default_a]
        (q_exact,p_exact) = make_sol(planewaves)

        # Initial data, and exact final result
        q0 = q_exact(  dt/2); p0 = p_exact(0)
        qf = q_exact(T+dt/2); pf = p_exact(T)

        # Additional parameters
        results_scheme = []
        for name,WaveH in WaveHs:
            q1,p1 = WaveH.Euler_p(q0,p0,dt,niter=Nt)
            results_ord = []
            for norm_p in (1,2,np.inf):
                err = norm(q1-qf,norm_p)/norm(q1,norm_p) # relative error
                results_ord.append(err)
            results_scheme.append(results_ord)
        results_ppw.append(results_scheme)
    results_Nx.append(results_ppw)

results = np.array(results_Nx) # Nx,ppw,scheme,ord

In [ ]:
colors = {"CrissCross2":"green","CrissCross4":"green",
          "Centered2":"blue","Centered4":"blue","Centered6":"blue",
          "Selling2":"red","Selling4":"red","Selling6":"red",
          "ConvSelling2":"orange","ConvSelling4":"orange","ConvSelling6":"orange",
          "SmoothSelling2":"purple","SmoothSelling4":"purple","SmoothSelling6":"purple"}
schemes6 = colors.keys()
schemes4 = [scheme for scheme in colors if scheme[-1] in ('2','4')]

- The Selling schemes are usually more accurate than the alternatives. Exceptions : 
  * low resolution, order 6. The Selling schemes are handicapped by their large stencils
  * order 6, high ppw, $L^\infty$ error with non-smooth coefficients, see below.
- The Selling variants with smooth coefficients, or with convolved coefficients, are more accurate at high ppw, high order, and in the $L^\infty$ error.  The difference disappears at low ppw, low order, or in the $L^2$ error. (Note that a little bit of tuning is required to get the correct with $\sigma$ and relaxation parameter $\epsilon$ of the deconvolution.)

**Note on the sixth order.**
The coefficients vary too quickly in the domain to really see the sixth order convergence at the considered domain resolutions.  
The Selling schemes are particularly sensitive to this, since they have very quite large stencils due to the combination of high anisotropy and high order. It is unclear wether using sixth order schemes is relevant in applications, given that the coefficients often have discontinuities. In addition, we only use a second order scheme in time, which becomes limiting at high ppw with a sixth order scheme in space.


In [ ]:
Nx=250; err_ord=np.inf
schemes = schemes4
iNx = Nxs.index(Nx) 
iord = {1:0,2:1,np.inf:2}[err_ord] 
for (name,_),error in zip(WaveHs,results[iNx,:,:,iord].T):
    if name not in schemes: continue
    plt.loglog(ppws,error,label=name,color=colors[name])
plt.title(f"Errors for {Nx=}, norm order={err_ord}")
plt.legend(loc="upper right"); plt.xlabel('ppw'); plt.ylabel('error');

More explicitly, the use of smooth coefficients reduces the error by approximately 50% in the most favorable case (fourth order, highest ppw).

In [ ]:
[name for (name,_) in WaveHs],results[iNx,-1,:,iord]

In [ ]:
Nx=250; err_ord=2
schemes = schemes4
iNx = Nxs.index(Nx) 
iord = {1:0,2:1,np.inf:2}[err_ord] 
for (name,_),error in zip(WaveHs,results[iNx,:,:,iord].T):
    if name not in schemes: continue
    plt.loglog(ppws,error,label=name,color=colors[name])
plt.title(f"Errors for {Nx=}, norm order={err_ord}")
plt.legend(loc="upper right"); plt.xlabel('ppw'); plt.ylabel('error');

In [ ]:
Nx=150; err_ord=2
schemes = schemes4
iNx = Nxs.index(Nx) 
iord = {1:0,2:1,np.inf:2}[err_ord] 
for (name,_),error in zip(WaveHs,results[iNx,:,:,iord].T):
    if name not in schemes: continue
    plt.loglog(ppws,error,label=name,color=colors[name])
plt.title(f"Errors for {Nx=}, norm order={err_ord}")
plt.legend(loc="upper right"); plt.xlabel('ppw'); plt.ylabel('error');

### 3.5 Elastic accuracy comparison

The Selling schemes are more accurate at low ppw and/or low order. 
For the highest ppw and order (ppw=20, order=4), the Lebedev scheme sometimes becomes more accurate. 
Possible explanations : 
- This could be due to the smaller (non-adaptive) stencil of the Lebedev scheme.
- This could be the time discretization error (The CFL condition of the lebedev scheme is less strict).

For mitigating the second point, we can either reduce the time step, or use a fourth order time integration scheme in time.


In [ ]:
%%time
# Problem parameters
Nxs = [150] # Considered domain resolutions
ppws = [7,8,10,12,15,20] # ppw in flat domain. 
Nt=100

ϵ_elastic = 0.02
ϕ_elastic = lambda X:ϕ_default(X,ϵ_elastic)
M_flat = np.eye(2)
C_flat = Hooke.olivine[0].extract_xz().rotate_by(π/12).hooke; C_flat/=np.max(C_flat)
cfl_mult = 0.5

results_Nx = []
for Nx in Nxs:
    # Build the domain, and coefficients
    X,dx = how.make_domain(Nx,2)
    M,C,S,make_sol = make_test_e(M_flat,C_flat,X,dx,ϕ_elastic)
    # Now, the Lebedev scheme. Since point density is doubled, we take this into account. 
    # In three dimensions, point density would be quadrupled (->replace sqrt(2) with 2**(2/3) in next line)
    Nx_Leb = int(np.round(Nx/np.sqrt(2)))
    X_Leb,dx_Leb = how.make_domain(Nx_Leb,2)
    _,X_10,X_01,_ = ec.shifted_grids(X_Leb,dx_Leb)
    M_Leb,C_Leb,S_Leb,_ = make_test_e(M_flat,C_flat,X_Leb,dx_Leb,ϕ=ϕ_elastic)
    _,_,_,make_sol_10   = make_test_e(M_flat,C_flat,X_10, dx_Leb,ϕ=ϕ_elastic)
    _,_,_,make_sol_01   = make_test_e(M_flat,C_flat,X_01, dx_Leb,ϕ=ϕ_elastic)
    

    # Build the hamiltonians
    stag2 = ec.staggered(dx_Leb,2,'Periodic')
    stag4 = ec.staggered(dx_Leb,4,'Periodic')
    λ,e = Eikonal.VoronoiDecomposition(C); sel_C = λ,Hooke.moffset(e)
    λ,e = ts.conv_decomp(*ts.deconv(C,σ=5,ϵ=5e-4,depth=2)); conv_C = λ,Hooke.moffset(e)
    λ,e = Eikonal.VoronoiDecomposition(C,smooth=True); smooth_C = λ,Hooke.moffset(e)
#    lin_D = ts.linear_decomp(D) # Non-positive for this anisotropy
    # Set a timestep
    WaveH = aw.ElasticHamiltonian_Sparse(M,C,dx,2,S) # TODO : sel_C
    dt = cfl_mult*WaveH.dt_max()
    T = Nt*dt
    WaveHs_e = [ # All Hamiltonians
        ("Selling2",WaveH),
        ("CorrSelling2",ec.SellingCorrelated2H_ext(M,sel_C, X,dx,2,S)),
        ("ConvSelling2",ec.SellingCorrelated2H_ext(M,conv_C,X,dx,2,S)),
        ("SmoothSelling2",ec.SellingCorrelated2H_ext(M,smooth_C,X,dx,2,S)),
        ("Lebedev2",ec.LebedevH2_ext(M_Leb,C_Leb,stag2,X_Leb,S_Leb)),
        ("Selling4",aw.ElasticHamiltonian_Sparse(M,C,dx,4,S)), # TODO : sel_C
        ("CorrSelling4",ec.SellingCorrelated2H_ext(M,sel_C, X,dx,4,S)),
        ("ConvSelling4",ec.SellingCorrelated2H_ext(M,conv_C,X,dx,4,S)),
        ("SmoothSelling4",ec.SellingCorrelated2H_ext(M,smooth_C,X,dx,4,S)),
        ("Lebedev4",ec.LebedevH2_ext(M_Leb,C_Leb,stag4,X_Leb,S_Leb)),
#		("Sellinv6",aw.ElasticHamiltonian_Sparse(M,C,dx,6,S)),
#		("CorrSelling6",ec.SellingCorrelated2H_ext(M,C,X,dx,6,S)),		
        ]
    results_ppw = []
    for ppw in ppws:
        print(f"{Nx=}, {ppw=}")
        planewaves = [planewave_with_spatial_frequency(Nx/ppw,planewave) for planewave in planewaves_default_e]
        # Exact colocated solution
        (q_exact,p_exact) = make_sol(planewaves)        
        sol_Col = (q_exact(  dt/2),p_exact(0),  
                   q_exact(T+dt/2),p_exact(T))
        # Exact solution on the Lebedev grids
        q_10,p_10 = make_sol_10(planewaves)
        q_01,p_01 = make_sol_01(planewaves)
        sol_Leb = np.array(((q_10(  dt/2),q_01(  dt/2)), (p_10(0),p_01(0)),
                            (q_10(T+dt/2),q_01(T+dt/2)), (p_10(T),p_01(T))))
		
        # Additional parameters
        results_scheme = []
        for name,WaveH in WaveHs_e:
            q0,p0,qf,pf = sol_Leb if name.startswith("Lebedev") else sol_Col # 
            q1,p1 = WaveH.Euler_p(q0,p0,dt,niter=Nt)
            results_ord = []
            for norm_p in (1,2,np.inf):
                err = norm(q1-qf,norm_p)/norm(q1,norm_p) # relative error
                results_ord.append(err)
            results_scheme.append(results_ord)
        results_ppw.append(results_scheme)
    results_Nx.append(results_ppw)

results_e = np.array(results_Nx) # Nx,ppw,scheme,ord

In [ ]:
colors_e = {"Lebedev2":"blue","Lebedev4":"blue",
            "Selling2":"red","Selling4":"red","Selling6":"red",
            "CorrSelling2":"pink","CorrSelling4":"pink","CorrSelling6":"pink",
            "ConvSelling2":"orange","ConvSelling4":"orange","ConvSelling6":"orange",
            "SmoothSelling2":"purple","SmoothSelling4":"purple","SmoothSelling6":"purple"}
schemes4_e = [scheme for scheme in colors_e if scheme[-1] in ('2','4')]

In [ ]:
Nx=150; err_ord=2
schemes = schemes4_e
iNx = Nxs.index(Nx)
iord = {1:0,2:1,np.inf:2}[err_ord] 
for (name,_),error in zip(WaveHs_e,results_e[iNx,:,:,iord].T):
    if name not in schemes: continue
    plt.loglog(ppws,error,label=name,color=colors_e[name])
plt.title(f"Errors for {Nx=}, norm order={err_ord}")
plt.legend(loc="upper right"); plt.xlabel('ppw'); plt.ylabel('error');

Another example, with a different material, milder domain deformations, and a smaller time step.

In [ ]:
%%time
# Problem parameters
Nxs = [150] # Considered domain resolutions
ppws = [7,8,10,12,15,20] # ppw in flat domain. 
Nt=100

ϵ_elastic = 0.01 # Smaller domain deformations
ϕ_elastic = lambda X:ϕ_default(X,ϵ_elastic)
M_flat = np.eye(2)
C_flat = Hooke.stishovite[0].extract_xz().rotate_by(π/8).hooke; C_flat/=np.max(C_flat)
cfl_mult = 0.2 # Small time step, otherwise time discretization error dominates

results_Nx = []
for Nx in Nxs:
    # Build the domain, and coefficients
    X,dx = how.make_domain(Nx,2)
    M,C,S,make_sol = make_test_e(M_flat,C_flat,X,dx,ϕ_elastic)
    # Now, the Lebedev scheme. Since point density is doubled, we take this into account. 
    # In three dimensions, point density would be quadrupled (->replace sqrt(2) with 2**(2/3) in next line)
    Nx_Leb = int(np.round(Nx/np.sqrt(2)))
    X_Leb,dx_Leb = how.make_domain(Nx_Leb,2)
    _,X_10,X_01,_ = ec.shifted_grids(X_Leb,dx_Leb)
    M_Leb,C_Leb,S_Leb,_ = make_test_e(M_flat,C_flat,X_Leb,dx_Leb,ϕ=ϕ_elastic)
    _,_,_,make_sol_10   = make_test_e(M_flat,C_flat,X_10, dx_Leb,ϕ=ϕ_elastic)
    _,_,_,make_sol_01   = make_test_e(M_flat,C_flat,X_01, dx_Leb,ϕ=ϕ_elastic)
    

    # Build the hamiltonians
    stag2 = ec.staggered(dx_Leb,2,'Periodic')
    stag4 = ec.staggered(dx_Leb,4,'Periodic')
    λ,e = Eikonal.VoronoiDecomposition(C); sel_C = λ,Hooke.moffset(e)
    #    smooth_D = Eikonal.VoronoiDecomposition(C,smooth=True)
    λ,e = ts.conv_decomp(*ts.deconv(C,σ=5,ϵ=5e-4,depth=2)); conv_C = λ,Hooke.moffset(e)
    λ,e = Eikonal.VoronoiDecomposition(C,smooth=True); smooth_C = λ,Hooke.moffset(e)
#    lin_D = ts.linear_decomp(D) # Non-positive for this anisotropy
    # Set a timestep
    WaveH = aw.ElasticHamiltonian_Sparse(M,C,dx,2,S) # TODO : sel_C
    dt = cfl_mult*WaveH.dt_max()
    T = Nt*dt
    WaveHs_e = [ # All Hamiltonians
        ("Selling2",WaveH),
        ("CorrSelling2",ec.SellingCorrelated2H_ext(M,sel_C, X,dx,2,S)),
        ("ConvSelling2",ec.SellingCorrelated2H_ext(M,conv_C,X,dx,2,S)),
        ("SmoothSelling2",ec.SellingCorrelated2H_ext(M,smooth_C,X,dx,2,S)),
        ("Lebedev2",ec.LebedevH2_ext(M_Leb,C_Leb,stag2,X_Leb,S_Leb)),
        ("Selling4",aw.ElasticHamiltonian_Sparse(M,C,dx,4,S)), # TODO : sel_C
        ("CorrSelling4",ec.SellingCorrelated2H_ext(M,sel_C, X,dx,4,S)),
        ("ConvSelling4",ec.SellingCorrelated2H_ext(M,conv_C,X,dx,4,S)),
        ("SmoothSelling4",ec.SellingCorrelated2H_ext(M,smooth_C,X,dx,4,S)),
        ("Lebedev4",ec.LebedevH2_ext(M_Leb,C_Leb,stag4,X_Leb,S_Leb)),
#		("Sellinv6",aw.ElasticHamiltonian_Sparse(M,C,dx,6,S)),
#		("CorrSelling6",ec.SellingCorrelated2H_ext(M,C,X,dx,6,S)),		
        ]
    results_ppw = []
    for ppw in ppws:
        print(f"{Nx=}, {ppw=}")
        planewaves = [planewave_with_spatial_frequency(Nx/ppw,planewave) for planewave in planewaves_default_e]
        # Exact colocated solution
        (q_exact,p_exact) = make_sol(planewaves)        
        sol_Col = (q_exact(  dt/2),p_exact(0),  
                   q_exact(T+dt/2),p_exact(T))
        # Exact solution on the Lebedev grids
        q_10,p_10 = make_sol_10(planewaves)
        q_01,p_01 = make_sol_01(planewaves)
        sol_Leb = np.array(((q_10(  dt/2),q_01(  dt/2)), (p_10(0),p_01(0)),
                            (q_10(T+dt/2),q_01(T+dt/2)), (p_10(T),p_01(T))))
		
        # Additional parameters
        results_scheme = []
        for name,WaveH in WaveHs_e:
            q0,p0,qf,pf = sol_Leb if name.startswith("Lebedev") else sol_Col # 
            q1,p1 = WaveH.Euler_p(q0,p0,dt,niter=Nt)
            results_ord = []
            for norm_p in (1,2,np.inf):
                err = norm(q1-qf,norm_p)/norm(q1,norm_p) # relative error
                results_ord.append(err)
            results_scheme.append(results_ord)
        results_ppw.append(results_scheme)
    results_Nx.append(results_ppw)

results_e = np.array(results_Nx) # Nx,ppw,scheme,ord

This time, the Selling schemes remain consistently ahead. Note that smooth coefficients (convolved, or obtained using the Selling Variant), are needed for optimal convergence.

In [ ]:
Nx=150; err_ord=2
schemes = schemes4_e
iNx = Nxs.index(Nx)
iord = {1:0,2:1,np.inf:2}[err_ord] 
for (name,_),error in zip(WaveHs_e,results_e[iNx,:,:,iord].T):
    if name not in schemes: continue
    plt.loglog(ppws,error,label=name,color=colors_e[name])
plt.title(f"Errors for {Nx=}, norm order={err_ord}")
plt.legend(loc="upper right"); plt.xlabel('ppw'); plt.ylabel('error');